This script is used to test for a three-way congruence x SNR x condition (precue,pretone) interaction for both RT and choice in the section:
"Rule- and stimulus-based biases had different effects on choices and RTs"

In [1]:
#clear memory
rm(list=ls())

## loading data/libraries ##

#load libraries
library(lme4)
#library(plyr)
library(dplyr)
#library(tidyr)
library(car)
library(lmerTest)
library(emmeans)
#library(afex)
emm_options(lmerTest.limit = 22676)
library(lattice)

switch(Sys.info()[['sysname']],
       Windows = PROJECT_DIR <- paste0('C:/Users/',Sys.getenv('USERNAME'),
                              '/Dropbox/Goldlab/AuditoryPriors/'),
       Darwin = PROJECT_DIR <- '~/Dropbox/Goldlab/AuditoryPriors/'
)

setwd(paste0(PROJECT_DIR,'data processing'))

DATA_FILE_PC = '../FitGDDM/data/priorOnly_28-Jan-2020.csv'
DATA_FILE_PT ='data/pretoneOnly_for_R_29-May-2020.csv'

Loading required package: Matrix

Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'car' was built under R version 4.0.4"
Loading required package: carData

Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


Attaching package: 'car'


The following object is masked from 'package:dplyr':

    recode


Warning message:
"package 'lmerTest' was built under R version 4.0.4"

Attaching package: 'lmerTest'


The following object is masked from 'package:lme4':

    lmer


The following object is masked from 'package:stats':

 

In [2]:
### load in precue data ###
pc_df <- read.table(DATA_FILE_PC,sep=',', header=TRUE, 
                    stringsAsFactors=FALSE,na.strings = 'NaN')

#remove missing data
pc_df <- subset(pc_df,!is.na(choice01) & !is.na(RT))


### load in pretone data ###
pt_df <- read.table(DATA_FILE_PT,sep=',', header=TRUE, 
                    stringsAsFactors=FALSE,na.strings = 'NaN')

#get rid of 0 pretones trials and missing trials in case this wasn't already done
pt_df <- arrange(subset(pt_df,pretoneLength > 0),subject,trialID)

#remove missing data
pt_df <- subset(pt_df,!is.na(choice01) & !is.na(RT))

head(pc_df)
head(pt_df)

,subject,optionName,cond,trialID,SNR,prior,isH,success,RT,choice01
,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>
1,AdL,priorOnly,7,1,-0.10,0,0,1,794.4652,0
2,AdL,priorOnly,7,2,-0.10,0,0,1,466.9621,0
3,AdL,priorOnly,7,3,0.20,0,1,0,440.8888,0
4,AdL,priorOnly,7,4,-0.10,0,0,1,546.5004,0
5,AdL,priorOnly,7,5,-0.50,0,0,0,662.5006,1
6,AdL,priorOnly,7,6,-0.05,0,0,1,663.1985,0


,subject,optionName,cond,trialID,SNR,prior,isH,success,RT,choice01,...,pretoneSeqisH_5,pretoneSeqisH_6,pretoneSeqisH_7,pretoneSeqisH_8,pretoneSeqisH_9,pretoneSeqisH_10,pretoneSeqisH_11,pretoneSeqisH_12,pretoneSeqisH_13,pretoneSeqisH_14
,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,AdR,pretoneOnly,3,1,-0.10,-2,0,1,817.7540,0,...,0,0,0,0,0,NA,NA,NA,NA,NA
2,AdR,pretoneOnly,3,2,-0.05,-2,0,0,876.3964,1,...,0,1,0,0,NA,NA,NA,NA,NA,NA
3,AdR,pretoneOnly,3,3,0.50,-2,1,1,873.0879,1,...,0,0,NA,NA,NA,NA,NA,NA,NA,NA
4,AdR,pretoneOnly,3,4,-0.20,0,0,1,778.5892,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,AdR,pretoneOnly,3,5,-0.10,-2,0,1,766.6472,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,AdR,pretoneOnly,3,6,0.50,2,1,1,659.4151,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [3]:
### set up precue data/variables

#compute abs SNR
pc_df$aSNR <- abs(pc_df$SNR)

#now restrict to high/low prior, min/max SNR trials
pc_df <- subset(pc_df,prior!=0 & aSNR %in% c(0.05,0.5))
unique(pc_df$prior)
unique(pc_df$aSNR)

#set up factors
# WARNING: THIS IS CONGRUENT WITH STIM! Doesn't matter if only analyzing correct RT but beware!!!
pc_df$congruent <- as.numeric((pc_df$prior==2 & pc_df$isH==1) | (pc_df$prior==-2 & pc_df$isH==0))
pc_df$congruent.fs <- factor(pc_df$congruent,levels=c(1,0),
                            labels=c("congruent","incongruent"))

[1] -2  2

[1] 0.50 0.05

In [4]:
### set up pretone data/variables

#compute abs SNR
pt_df$aSNR <- abs(pt_df$SNR)

#now restrict to min/max SNR trials
pt_df <- subset(pt_df,aSNR %in% c(0.05,0.5))
unique(pt_df$aSNR)

#set up congruent proxy variable
# WARNING: THIS IS CONGRUENT WITH STIM! Doesn't matter if only analyzing correct RT but beware!!!
pt_df$congruent <- NA
attach(pt_df)
#setting these differently from pretone_RT_lm for congruence w/ precue and easy removal of mixed pretones
pt_df$congruent[(isH==1 & pretoneSeqisH_1==1 & pretoneSeqisH_2==1) | 
           (isH==0 & pretoneSeqisH_1==0 & pretoneSeqisH_2==0)] <- 1
pt_df$congruent[(isH==1 & pretoneSeqisH_1==1 & pretoneSeqisH_2==0) | 
        (isH==0 & pretoneSeqisH_1==0 & pretoneSeqisH_2==1)] <-  -1 
pt_df$congruent[(isH==1 & pretoneSeqisH_1==0 & pretoneSeqisH_2==1) | 
        (isH==0 & pretoneSeqisH_1==1 & pretoneSeqisH_2==0)] <- -1
pt_df$congruent[(isH==0 & pretoneSeqisH_1==1 & pretoneSeqisH_2==1) | 
                  (isH==1 & pretoneSeqisH_1==0 & pretoneSeqisH_2==0)] <- 0
detach(pt_df)
#very minor gut check
if (any(is.na(pt_df$congruent))) {
  warning('Congruent conditions improperly assigned!')
}


#now remove trials that are not HH/LL (this is a little circuitous but lets me crib from the pretone_RT_lm code)
pt_df <- subset(pt_df,congruent %in% c(0,1))

con_check <- arrange(unique(pt_df[,c('congruent','isH','pretoneSeqisH_1','pretoneSeqisH_2')]),
                     congruent,isH)
con_check

#set up factors
# WARNING: THIS IS CONGRUENT WITH STIM! Doesn't matter if only analyzing correct RT but beware!!!
pt_df$congruent.fs <- factor(pt_df$congruent,levels=c(1,0),
                            labels=c("congruent","incongruent"))

[1] 0.05 0.50

congruent,isH,pretoneSeqisH_1,pretoneSeqisH_2
<dbl>,<int>,<int>,<int>
0,0,1,1
0,1,0,0
1,0,0,0
1,1,1,1


## Choice analysis
Is there a 3-way congruent x |SNR| x condition interaction in the choice behavior?

Note that to analyze congruence in choice, we use accuracy coding instead of choice coding, as this is equivalent to flipping the low freq pretone choices.

In [5]:
#merge

keep_vars <- c('subject','optionName','trialID','aSNR','congruent','congruent.fs','success')
pct_df <- rbind(pc_df[,keep_vars],pt_df[,keep_vars])
head(pct_df)
unique(pct_df$optionName)

,subject,optionName,trialID,aSNR,congruent,congruent.fs,success
,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<int>
51,AdL,priorOnly,51,0.50,1,congruent,1
55,AdL,priorOnly,55,0.50,1,congruent,1
58,AdL,priorOnly,58,0.05,1,congruent,1
59,AdL,priorOnly,59,0.50,0,incongruent,1
61,AdL,priorOnly,61,0.05,1,congruent,1
62,AdL,priorOnly,62,0.50,1,congruent,1


[1] "priorOnly"   "pretoneOnly"

In [6]:
#set up variables/contrasts
pct_df$aSNR.fs <- factor(pct_df$aSNR,levels=c(0.5,0.05),
                            labels=c("high","low"))
contrasts(pct_df$aSNR.fs) <- contr.sum(2)
contrasts(pct_df$aSNR.fs)

pct_df$block.fs <- factor(pct_df$optionName,levels=c('pretoneOnly','priorOnly'))
contrasts(pct_df$block.fs) <- contr.sum(2)
contrasts(pct_df$block.fs)

#seems like factor coding gets lost after rbind...phooey
contrasts(pct_df$congruent.fs) <- contr.sum(2)
contrasts(pct_df$congruent.fs)

#extract for zero-corr model
pct_df[,c('aSNR.fs1')] <- 
  model.matrix(~1+pct_df$aSNR.fs,pct_df)[,2]
unique(pct_df[,c('aSNR','aSNR.fs','aSNR.fs1')])

pct_df[,c('block.fs1')] <- 
  model.matrix(~1+pct_df$block.fs,pct_df)[,2]
unique(pct_df[,c('optionName','block.fs','block.fs1')])

pct_df[,c('congruent.fs1')] <- 
  model.matrix(~1+pct_df$congruent.fs,pct_df)[,2]
unique(pct_df[,c('congruent','congruent.fs','congruent.fs1')])

high,1
low,-1


pretoneOnly,1
priorOnly,-1


congruent,1
incongruent,-1


,aSNR,aSNR.fs,aSNR.fs1
,<dbl>,<fct>,<dbl>
51,0.50,high,1
58,0.05,low,-1


,optionName,block.fs,block.fs1
,<chr>,<fct>,<dbl>
51,priorOnly,priorOnly,-1
3,pretoneOnly,pretoneOnly,1


,congruent,congruent.fs,congruent.fs1
,<dbl>,<fct>,<dbl>
51,1,congruent,1
59,0,incongruent,-1


### congruency interaction model

In [7]:
#full model
ccon.lm.0 <- glmer(success~aSNR.fs*congruent.fs*block.fs + (1+aSNR.fs*congruent.fs*block.fs|subject), 
                 data=pct_df,family=binomial, 
                 control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see ?isSingular



In [8]:
#zero corr model converges
#FINAL
ccon.lm.0nc <- glmer(success~aSNR.fs*congruent.fs*block.fs + (1+aSNR.fs1*congruent.fs1*block.fs1||subject), 
                 data=pct_df,family=binomial, 
                 control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

In [14]:
#FINAL

Anova(ccon.lm.0nc,type="III")
summary(ccon.lm.0nc)

,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>
(Intercept),257.3059752,1,6.633775e-58
aSNR.fs,285.2384973,1,5.420930e-64
congruent.fs,141.1149396,1,1.518479e-32
block.fs,0.2257891,1,6.346639e-01
aSNR.fs:congruent.fs,57.4028049,1,3.551005e-14
aSNR.fs:block.fs,2.6967650,1,1.005521e-01
congruent.fs:block.fs,65.8283262,1,4.919607e-16
aSNR.fs:congruent.fs:block.fs,10.8166249,1,1.005927e-03


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: success ~ aSNR.fs * congruent.fs * block.fs + (1 + aSNR.fs1 *  
    congruent.fs1 * block.fs1 || subject)
   Data: pct_df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
 11854.6  11978.0  -5911.3  11822.6    16471 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-11.0738   0.0694   0.1706   0.4308   3.1080 

Random effects:
 Groups    Name                             Variance Std.Dev.
 subject   (Intercept)                      0.45875  0.6773  
 subject.1 aSNR.fs1                         0.35254  0.5937  
 subject.2 congruent.fs1                    0.15476  0.3934  
 subject.3 block.fs1                        0.02218  0.1489  
 subject.4 aSNR.fs1:congruent.fs1           0.09374  0.3062  
 subject.5 aSNR.fs1:block.fs1               0.03023  0.1739  
 subject.6 con

## RT analysis

Is there a 3-way congruent x |SNR| x condition interaction in RTs?

In [21]:
# compute medians for analysis and merge data [restricting to correct trials]

pc_df.med <- summarise(group_by(subset(pc_df,success==1),
                                subject,optionName,aSNR,isH,prior,congruent,congruent.fs),RT=median(RT))
pt_df.med <- summarise(group_by(subset(pt_df,success==1),
                                subject,optionName,aSNR,congruent,congruent.fs,isH,pretoneSeqisH_1,pretoneSeqisH_2),
                        RT=median(RT))

#keep_vars <- c('subject','optionName','trialID','SNR','aSNR','isH','congruent','congruent.fs','success','RT','choice01')
keep_vars <- c('subject','optionName','aSNR','isH','congruent','congruent.fs','RT')
pct_df.med <- rbind(pc_df.med[,keep_vars],pt_df.med[,keep_vars])
head(pct_df.med)
unique(pct_df.med$optionName)

`summarise()` regrouping output by 'subject', 'optionName', 'aSNR', 'isH', 'prior', 'congruent' (override with `.groups` argument)

`summarise()` regrouping output by 'subject', 'optionName', 'aSNR', 'congruent', 'congruent.fs', 'isH', 'pretoneSeqisH_1' (override with `.groups` argument)



subject,optionName,aSNR,isH,congruent,congruent.fs,RT
<chr>,<chr>,<dbl>,<int>,<dbl>,<fct>,<dbl>
AdL,priorOnly,0.05,0,1,congruent,557.0204
AdL,priorOnly,0.05,0,0,incongruent,1049.4183
AdL,priorOnly,0.05,1,0,incongruent,449.0412
AdL,priorOnly,0.05,1,1,congruent,370.9620
AdL,priorOnly,0.50,0,1,congruent,530.8959
AdL,priorOnly,0.50,0,0,incongruent,827.8666


[1] "priorOnly"   "pretoneOnly"

In [22]:
#set up variables

pct_df.med$aSNR.fs <- factor(pct_df.med$aSNR,levels=c(0.5,0.05),
                            labels=c("high","low"))
contrasts(pct_df.med$aSNR.fs) <- contr.sum(2)
contrasts(pct_df.med$aSNR.fs)

pct_df.med$block.fs <- factor(pct_df.med$optionName,levels=c('pretoneOnly','priorOnly'))
contrasts(pct_df.med$block.fs) <- contr.sum(2)
contrasts(pct_df.med$block.fs)

#seems like factor coding gets destroyed afer rbind...
contrasts(pct_df.med$congruent.fs) <- contr.sum(2)
contrasts(pct_df.med$congruent.fs)

#extract for zero-corr model
pct_df.med[,c('aSNR.fs1')] <- 
  model.matrix(~1+pct_df.med$aSNR.fs,pct_df.med)[,2]
unique(pct_df.med[,c('aSNR','aSNR.fs','aSNR.fs1')])

pct_df.med[,c('block.fs1')] <- 
  model.matrix(~1+pct_df.med$block.fs,pct_df.med)[,2]
unique(pct_df.med[,c('optionName','block.fs','block.fs1')])

pct_df.med[,c('congruent.fs1')] <- 
  model.matrix(~1+pct_df.med$congruent.fs,pct_df.med)[,2]
unique(pct_df.med[,c('congruent','congruent.fs','congruent.fs1')])


high,1
low,-1


pretoneOnly,1
priorOnly,-1


congruent,1
incongruent,-1


aSNR,aSNR.fs,aSNR.fs1
<dbl>,<fct>,<dbl>
0.05,low,-1
0.50,high,1


optionName,block.fs,block.fs1
<chr>,<fct>,<dbl>
priorOnly,priorOnly,-1
pretoneOnly,pretoneOnly,1


congruent,congruent.fs,congruent.fs1
<dbl>,<fct>,<dbl>
1,congruent,1
0,incongruent,-1


### congruency interaction model

In [23]:
#full model
rtccon.lm.0 <- lmer(RT~aSNR.fs*congruent.fs*block.fs + (1+aSNR.fs*congruent.fs*block.fs|subject), 
                 data=pct_df.med, 
                 control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see ?isSingular



In [24]:
#zero corr model
rtccon.lm.0nc <- lmer(RT~aSNR.fs*congruent.fs*block.fs + (1+aSNR.fs1*congruent.fs1*block.fs1||subject), 
                    data=pct_df.med, 
                    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see ?isSingular



In [25]:
#no 3-way, w/ corr (nope)
rtccon.lm.1 <- lmer(RT~aSNR.fs1*congruent.fs*block.fs + 
                      (1+aSNR.fs*congruent.fs + congruent.fs*block.fs + aSNR.fs*block.fs|subject), 
                    data=pct_df.med, 
                    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see ?isSingular



In [26]:
#FINAL
#no 3-way, zero corr converges!
rtccon.lm.1nc <- lmer(RT~aSNR.fs*congruent.fs*block.fs + 
                      (1+aSNR.fs1*congruent.fs1 + congruent.fs1*block.fs1 + aSNR.fs1*block.fs1||subject), 
                    data=pct_df.med, 
                    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

In [36]:
#FINAL (report whichever of these you reported elsewhere---I think the car::Anova)

Anova(rtccon.lm.1nc,test.statistic = "F",type="III")
#anova(rtccon.lm.1nc,type="III")

summary(rtccon.lm.1nc)

,F,Df,Df.res,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1779.810237,1,48.79954,4.603624e-40
aSNR.fs,74.781246,1,48.68139,2.097209e-11
congruent.fs,48.963248,1,47.83592,7.549891e-09
block.fs,161.491291,1,45.20722,1.591030e-16
aSNR.fs:congruent.fs,42.934027,1,47.44030,3.756215e-08
aSNR.fs:block.fs,1.563197,1,45.71000,2.175606e-01
congruent.fs:block.fs,52.659135,1,47.04159,3.323100e-09
aSNR.fs:congruent.fs:block.fs,6.538140,1,428.46397,1.090214e-02


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
RT ~ aSNR.fs * congruent.fs * block.fs + (1 + aSNR.fs1 * congruent.fs1 +  
    congruent.fs1 * block.fs1 + aSNR.fs1 * block.fs1 || subject)
   Data: pct_df.med
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

REML criterion at convergence: 8888.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.9333 -0.4586 -0.0668  0.3340  4.3396 

Random effects:
 Groups    Name                    Variance Std.Dev.
 subject   (Intercept)             12382.25 111.276 
 subject.1 aSNR.fs1                 2078.05  45.586 
 subject.2 congruent.fs1             414.02  20.348 
 subject.3 block.fs1                2820.01  53.104 
 subject.4 aSNR.fs1:congruent.fs1     62.37   7.898 
 subject.5 congruent.fs1:block.fs1   189.35  13.760 
 subject.6 aSNR.fs1:block.fs1        349.29  18.689 
 Residual                           8040.83  89.671 
Number of obs: 723, groups:  subje